# Interactive circos-like plots with Bokeh

<u>Prototyping for proteomic and phosphoproteomic datasets.</u> <br>
<b>Goals:</b> Clickable links to protein descriptions, ability to show multiple (large) datasets<br>
<b>Open questions:</b> How to represent proteins with multiple peptides/phosphorylations? How to categorize the proteins (which ontology?)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import TapTool, OpenURL, ColumnDataSource

import numpy as np
import pandas as pd

In [ ]:
# this line necessary to view the figure in the notebook
output_notebook() 

In [ ]:
# acquiring the data
# ID, chromosome, location, intensity
# index by chromosome, location in pandas? 

file = "pro_test_data.csv"
data = pd.read_csv(file)

# select only the top most intense peptide for the protein. Is this the best way? Will likely change.
data = data.iloc[data.groupby(['Protein Group Accessions']).apply(lambda x: x['Intensity'].idxmax())]
data = data
data.set_index('Protein Group Accessions')

The following code is based on the burtin example in the Bokeh gallery: http://bokeh.pydata.org/en/latest/docs/gallery/burtin.html

In [ ]:
# setting dimensions for the figure, as well as for the inner/outer radii
width = 800
height = 800

inner_radius = 150
outer_radius = 350

minr = np.sqrt(np.log(.001 * 1E4))
maxr = np.sqrt(np.log(1000000 * 1E4))
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic):
    return a * np.sqrt(np.log(mic * 1000000)) + b

big_angle = 2.0 * np.pi / (len(data) + 1)
small_angle = big_angle / 7
angles = np.pi/2 - big_angle/2 - data.index.to_series()*big_angle

In [ ]:
# plotting the figure
data_s = ColumnDataSource(data=dict(
    intensity=data.Intensity,
    ID = data['Protein Group Accessions']
    ))

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="black", background_fill_color="white",
    border_fill_color="white", tools = "wheel_zoom, crosshair, tap, pan, reset")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.annular_wedge(x=0, 
                y=0, 
                outer_radius=outer_radius, 
                inner_radius=rad(data.Intensity/100000000), 
                start_angle=-big_angle+angles+1*small_angle, 
                end_angle=-big_angle+angles+2*small_angle,
                color='#4daf4a',
                source=data_s
                )

# configure the tap tool
url = "http://www.uniprot.org/uniprot/@ID"
taptool = p.select(dict(type=TapTool))
taptool.callback = OpenURL(url=url)

In [ ]:
show (p)

Seems to work! But the dataset is pretty large, and it's hard to click on the right glyph. 